Install dependencies

In [1]:
!pip install google-generativeai requests python-dotenv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Load environment variables

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

Initialize Gemini

In [3]:
import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

model = genai.GenerativeModel("gemini-2.5-flash")

c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Quick Gemini test

In [4]:
response = model.generate_content("Hello there")
print(response.text)

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 26.075410397s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 26
}
]

GitHub file extractor

In [ ]:
import requests
import base64

def get_default_branch(repo):
    url = f"https://api.github.com/repos/{repo}"
    r = requests.get(url)
    r.raise_for_status()
    return r.json()["default_branch"]

def get_github_file(repo, path, branch=None):
    if branch is None:
        branch = get_default_branch(repo)

    url = f"https://api.github.com/repos/{repo}/contents/{path}?ref={branch}"
    r = requests.get(url)

    if r.status_code == 404:
        raise Exception(f"File not found: {repo}/{path} on branch {branch}")

    r.raise_for_status()
    data = r.json()
    return base64.b64decode(data["content"]).decode("utf-8")



Test GitHub extraction

In [ ]:
code = get_github_file("psf/requests", "src/requests/__init__.py")
print(code[:500])

#   __
#  /__)  _  _     _   _ _/   _
# / (   (- (/ (/ (- _)  /  _)
#          /

"""
Requests HTTP Library
~~~~~~~~~~~~~~~~~~~~~

Requests is an HTTP library, written in Python, for human beings.
Basic GET usage:

   >>> import requests
   >>> r = requests.get('https://www.python.org')
   >>> r.status_code
   200
   >>> b'Python is a programming language' in r.content
   True

... or POST:

   >>> payload = dict(key1='value1', key2='value2')
   >>> r = requests.post('https://httpbin.org/post', 


Decide the language

In [ ]:
def detect_language(path):
    if path.endswith(".py"):
        return "python"
    if path.endswith(".ts"):
        return "typescript"
    if path.endswith(".js"):
        return "javascript"
    return "unknown"

Prompt for documentation generation

In [ ]:
def add_documentation(code, language):
    prompt = f"""
You are a senior {language} developer.

Task:
Add clear, professional documentation to the following source code.

Rules:
- Do NOT change logic
- Add docstrings / comments where appropriate
- Keep formatting clean
- Follow best practices for {language}

Source code:
{code}
"""
    response = model.generate_content(prompt)
    return response.text


Prompt for test generation

In [ ]:
def generate_tests(code, language):
    prompt = f"""
You are a senior {language} developer.

Task:
Write unit tests for the following source code.

Rules:
- Use best testing framework for {language}
- Cover main logic and edge cases
- Return ONLY test code
- Do not explain anything

Source code:
{code}
"""
    response = model.generate_content(prompt)
    return response.text

Full agent pipeline

In [ ]:
def document_and_test(repo, path):
    code = get_github_file(repo, path)
    language = detect_language(path)

    documented_code = add_documentation(code, language)
    tests = generate_tests(code, language)

    return {
        "original": code,
        "documented": documented_code,
        "tests": tests
    }

In [ ]:
result = document_and_test(
    "pallets/flask",
    "src/flask/cli.py"
)

print("===== DOCUMENTED CODE =====")
print(result["documented"][:2000])

print("\n===== GENERATED TESTS =====")
print(result["tests"][:2000])


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash
Please retry in 42.885193608s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
}
, retry_delay {
  seconds: 42
}
]

Save results to files

In [ ]:
def save_output(path, documented_code, tests):
    base = path.split("/")[-1].replace(".py", "")
    
    with open(f"{base}_documented.py", "w", encoding="utf-8") as f:
        f.write(documented_code)

    with open(f"test_{base}.py", "w", encoding="utf-8") as f:
        f.write(tests)
        
save_output(
    "cli.py",
    result["documented"],
    result["tests"]
)